In [1]:
%cd capstone-project-9900w12achatllm

/gemini/code/capstone-project-9900w12achatllm


In [2]:
!pip install -e ".[torch,metrics]"
!pip install --upgrade pandas pyarrow datasets
!pip install auto_gptq>=0.5.0
!pip install optimum
!pip uninstall apex -y

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///gemini/code/capstone-project-9900w12achatllm
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/20/5c/244db59e074e80248fdfa60495eeee257e4d97c3df3487df68be30cd60c8/transformers-4.42.3-py3-none-any.whl (9.3 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/19/99/c5e0292a6d2a62e95c3dfe674ce9e8f8a9fe5d4835d3c9bb9b3e016f02ae/peft-0.11.1-py3-none-any.whl (251 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 17.8 MB/s

batch size = 3, max_example = 500, lr = 1e-5

In [3]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 1e-05 \
    --num_train_epochs 3.0 \
    --max_samples 500 \
    --per_device_train_batch_size 3 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-25_lr=1e-5 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 10 \
    --per_device_eval_batch_size 2

[2024-07-11 10:41:08,062] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 10:41:13 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:41:13,333 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:41:13,333 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:41:13,333 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:41:13,333 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:41:13,333 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:41:13,333 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 10:41:13,729 >> Special tokens have been added in the vocabulary, 

In [4]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/eval_2024-07-09-00-26-34_lr=1e-5 \
    --do_predict True \
    --adapter_name_or_path saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-25_lr=1e-5

[2024-07-11 10:59:33,978] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 10:59:38 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:59:38,848 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:59:38,848 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:59:38,848 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:59:38,848 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:59:38,848 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 10:59:38,848 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 10:59:39,377 >> Special tokens have been added in the vocabulary, make sure

batch size = 3, max_exmaple = 500, _lr=1e-4

In [5]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 1e-04 \
    --num_train_epochs 3.0 \
    --max_samples 500 \
    --per_device_train_batch_size 3 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-26_lr=1e-4 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 10 \
    --per_device_eval_batch_size 2

[2024-07-11 11:04:40,672] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 11:04:45 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:04:45,260 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:04:45,260 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:04:45,260 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:04:45,260 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:04:45,260 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:04:45,260 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 11:04:45,555 >> Special tokens have been added in the vocabulary, 

In [6]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/eval_2024-07-09-00-26-35_lr=1e-4 \
    --do_predict True \
    --adapter_name_or_path saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-26_lr=1e-4

[2024-07-11 11:20:20,680] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 11:20:25 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:20:25,140 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:20:25,140 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:20:25,140 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:20:25,140 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:20:25,140 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:20:25,140 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 11:20:25,471 >> Special tokens have been added in the vocabulary, make sure

batch size = 3, max_example = 500, lr=5e-4

In [7]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-04 \
    --num_train_epochs 3.0 \
    --max_samples 500 \
    --per_device_train_batch_size 3 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-27_lr=5e-4 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 10 \
    --per_device_eval_batch_size 2

[2024-07-11 11:24:41,046] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 11:24:45 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:24:45,446 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:24:45,446 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:24:45,446 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:24:45,446 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:24:45,446 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:24:45,446 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 11:24:45,720 >> Special tokens have been added in the vocabulary, 

In [8]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/eval_2024-07-09-00-26-36_lr=5e-4 \
    --do_predict True \
    --adapter_name_or_path saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-27_lr=5e-4

[2024-07-11 11:40:23,069] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 11:40:27 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:40:27,628 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:40:27,628 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:40:27,628 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:40:27,628 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:40:27,629 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 11:40:27,629 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 11:40:28,101 >> Special tokens have been added in the vocabulary, make sure